<a href="https://colab.research.google.com/github/Arnab515/portfolio/blob/main/trade_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

def getTickerPrice(ticker: str, date: datetime) -> float:
    # Dummy function to simulate getting the price of a security at a given date
    return random.uniform(1, 100)

def calculate_trade_performance(trades: pd.DataFrame) -> pd.Series:
    # Handling edge cases
    if trades.empty:
        return pd.Series({})

    # Fill missing 'Size' with 1
    trades['Size'].fillna(1, inplace=True)

    # Initializing metrics
    total_trades = len(trades)
    total_buy_trades = len(trades[trades['Side'] == 'buy'])
    total_sell_trades = len(trades[trades['Side'] == 'sell'])
    total_volume_traded = trades['Size'].sum()

    # Calculate gross profit, gross loss, net profit
    trades['MarketPrice'] = trades.apply(lambda row: getTickerPrice(row['Symbol'], row['Date']), axis=1)
    trades['PnL'] = np.where(trades['Side'] == 'buy',
                             (trades['MarketPrice'] - trades['Price']) * trades['Size'],
                             (trades['Price'] - trades['MarketPrice']) * trades['Size'])
    gross_profit = trades[trades['PnL'] > 0]['PnL'].sum()
    gross_loss = trades[trades['PnL'] < 0]['PnL'].sum()
    net_profit = gross_profit + gross_loss  # Since gross_loss is negative

    # Calculate winning and losing trades
    winning_trades = len(trades[trades['PnL'] > 0])
    losing_trades = len(trades[trades['PnL'] < 0])
    win_rate = winning_trades / total_trades if total_trades > 0 else 0

    # Collecting metrics in a dictionary
    metrics = {
        'Total Trades': total_trades,
        'Total Buy Trades': total_buy_trades,
        'Total Sell Trades': total_sell_trades,
        'Total Volume Traded': total_volume_traded,
        'Gross Profit': gross_profit,
        'Gross Loss': gross_loss,
        'Net Profit': net_profit,
        'Winning Trades': winning_trades,
        'Losing Trades': losing_trades,
        'Win Rate': win_rate
    }

    return pd.Series(metrics)

# Example usage
# Example DataFrame
data = {
    'Date': [datetime(2023, 1, 1), datetime(2023, 1, 2), datetime(2023, 1, 3)],
    'Symbol': ['AAPL', 'GOOG', 'MSFT'],
    'Side': ['buy', 'sell', 'buy'],
    'Size': [10, 5, 15],
    'Price': [150, 1200, 250]
}

trades_df = pd.DataFrame(data)
performance_metrics = calculate_trade_performance(trades_df)
print(performance_metrics)